In [1]:
# @title Install components
!curl https://ollama.ai/install.sh | sh
!pip install ollama

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  54821      0 --:--:-- --:--:-- --:--:-- 54880
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/

In [2]:
# @title Start server
import subprocess
proccess = subprocess.Popen(['ollama', 'serve'])

In [3]:

# @title Select model
model = "qwen3:14b"
!ollama pull {model}

# loading qa_data

In [4]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [5]:
from datasets import load_dataset


ds = load_dataset("pminervini/HaluEval", "qa")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

data-00000-of-00001.parquet:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

Generating data split:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Baseline prompt strategy

In [6]:
# def build_baseline_prompt(knowledge, question, answer):
#     return f"""
# You are an *expert hallucination detector* for LLM outputs.

# INSTRUCTIONS
# ------------
# For each case you will receive three fields:
# <<<DELIMITER
# "knowledge": factual statements
# "question": original user question
# "response": model answer to assess
# DELIMITER>>>

# Your task:
# 1. Decide whether the *response* is fully supported by the *knowledge* given the *question*.
#    • Output **0**  → non-hallucinated (fully supported)
#    • Output **1**  → hallucinated (contains any unsupported claim)
# 2. Provide a brief reason (1-2 sentences).
# 3. **Return ONLY a valid JSON object** with the exact keys **"label"** and **"reason"**.

# OUTPUT SCHEMA (must be strictly followed)
# ```json
# {{
#   "Final Answer": 0 or 1,
#   "reason": "<brief justification>"
# }}
# """


In [7]:
def build_baseline_prompt(knowledge, question, answer):
    return f"""
You are an expert hallucination detector. Analyze the given answer based on the provided knowledge and question.

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
A: Pride and Prejudice was written by Jane Austen.
Final Answer: "0"

example2:
Q: What is the capital city of Mars?
A: The capital of Mars is Olympus City.
Final Answer: "1"

now is your turn:
Q: {question}
A: {answer}

At the end, provide your final judgment "1" or "0" firstly:
Final Answer: "1" if the answer contains hallucination, otherwise "0".

If Final Answer is "0", explain briefly why the answer is factually correct.
The final answer can only be " Final Answer: "1" " or " Final Answer: "0" "
"""


In [8]:
data = ds['data']

In [9]:
samples = data.shuffle(seed=42).select(range(250))

In [10]:
import random, requests, json, re, time
from collections import Counter

URL = "http://localhost:11434/api/generate"
HEADERS = {"Content-Type": "application/json"}

# 记录混淆矩阵
cm = Counter(TP=0, FP=0, TN=0, FN=0)

def call_llm(prompt: str) -> int:
    """调用本地模型并解析 'Final Answer: 0/1'，返回 0 或 1。"""
    resp = requests.post(URL, headers=HEADERS,
                         data=json.dumps({"model": "qwen3:14b",
                                          "prompt": prompt,
                                          "stream": False}))
    result = resp.json()["response"]
    m = re.search(r'Final\s*Answer\s*[:：]\s*"?([01])"?', result, re.IGNORECASE)
    if not m:
        raise ValueError(f"无法解析模型输出:\n{result}")
    return int(m.group(1))

start = time.perf_counter()

for i, item in enumerate(samples, 1):
    k, q = item["knowledge"], item["question"]

    # ① 先测 hallucinated_answer，期望标签 = 1
    pred_h = call_llm(build_baseline_prompt(k, q, item["hallucinated_answer"]))
    label_h = 1
    if   label_h and pred_h: cm["TP"] += 1
    elif label_h and not pred_h: cm["FN"] += 1
    elif not label_h and not pred_h: cm["TN"] += 1
    else: cm["FP"] += 1

    # ② 再测 right_answer，期望标签 = 0
    pred_r = call_llm(build_baseline_prompt(k, q, item["right_answer"]))
    label_r = 0
    if   label_r and pred_r: cm["TP"] += 1
    elif label_r and not pred_r: cm["FN"] += 1
    elif not label_r and not pred_r: cm["TN"] += 1
    else: cm["FP"] += 1

    print(f"\n====== Sample {i} ======")
    print(f"Hallucinated → pred={pred_h}  (should be 1)")
    print(f"Ground-truth  → pred={pred_r}  (should be 0)")

# -------- 计时结束 --------
elapsed = time.perf_counter() - start

# ========== 计算指标 ==========
TP, FP, TN, FN = cm["TP"], cm["FP"], cm["TN"], cm["FN"]

precision = TP / (TP + FP) if TP + FP else 0
recall    = TP / (TP + FN) if TP + FN else 0
f1        = 2*precision*recall / (precision + recall) if precision + recall else 0
accuracy  = (TP + TN) / (TP + TN + FP + FN)

print("\n=== Confusion Matrix ===")
print(cm)
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"Accuracy : {accuracy:.4f}")
print("*"*40)
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")


====== Sample 1 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 2 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 3 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 4 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=1  (should be 0)

====== Sample 5 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 6 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=1  (should be 0)

====== Sample 7 ======
Hallucinated → pred=0  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 8 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== Sample 9 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=1  (should be 0)

====== Sample 10 ======
Hallucinated → pred=1  (should be 1)
Ground-truth  → pred=0  (should be 0)

====== S

In [11]:
# import json, re, requests, random

# def extract_final_answer(text: str):
#     """
#     从模型完整输出中解析 label（0/1）与 reason。
#     兼容:
#       • <think> ... </think> 链式思考
#       • ```json ... ``` / ``` ... ``` 代码块
#       • key 名叫 "Final Answer" 或 "label"
#     """
#     # ① 去掉 <think> 块（非贪婪）
#     text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL|re.IGNORECASE)

#     # ② 抓 ```json ... ```，若不存在再抓普通 ```
#     m = re.search(r"```json\s*(\{.*?\})\s*```", text, re.DOTALL)
#     if not m:
#         m = re.search(r"```\s*(\{.*?\})\s*```", text, re.DOTALL)

#     # ③ 若仍抓不到，兜底取第一个 {...}
#     if not m:
#         m = re.search(r"\{[^{}]*\}", text, re.DOTALL)

#     if not m:
#         raise ValueError("❌ 未找到 JSON！原始输出:\n" + text)

#     json_str = m.group(1)

#     # ④ 常见修复：单引号→双引号，尾逗号→空
#     json_str = (
#         json_str.replace("'", '"')
#                 .replace(",}", "}")
#                 .replace(",]", "]")
#                 .strip()
#     )

#     obj = json.loads(json_str)                 # ⑤ 直接反序列化；出错会抛异常

#     # ⑥ 提取 label 与 reason
#     if "Final Answer" in obj:
#         label = int(obj["Final Answer"])
#     elif "label" in obj:
#         label = int(obj["label"])
#     else:
#         raise KeyError("JSON 中缺少 'Final Answer' 或 'label' 字段！")

#     reason = obj.get("reason", "")
#     return label, reason


# # =========== 你的主循环改写 =============
# count1, count0, failed = 0, 0, 0

# for i, item in enumerate(samples, 1):
#     knowledge = item['knowledge']
#     question  = item['question']
#     answer    = item['hallucinated_answer']   # 或 'right_answer'

#     prompt = build_baseline_prompt(knowledge, question, answer)

#     resp = requests.post(
#         "http://localhost:11434/api/generate",
#         headers={"Content-Type": "application/json"},
#         json={          # 用 json= 而不是 data=+dumps 省一步
#             "model": "qwen3:14b",
#             "prompt": prompt,
#             "stream": False
#         },
#         timeout=120     # 视情况加个超时
#     )

#     result_text = resp.json()["response"]
#     # print("000000000000000000000000000000000000000000000000000")
#     # print(f"Model Output:\n{result_text}")
#     print(f"\n====== Sample {i+1} ======")

#     try:
#         label, reason = extract_final_answer(result_text)
#         if label == 1:
#             count1 += 1
#         elif label == 0:
#             count0 += 1
#     except Exception as e:
#         failed += 1
#         print(f"\n⚠️ 解析失败 (sample {i}) — {e}\n{result_text}\n")

# print(f"\n✔️ 统计结果：1 → {count1}  | 0 → {count0}  | 解析失败 → {failed}")


In [12]:
# import random
# import requests
# import json
# import re

# count = 0
# count0 = 0
# for i, item in enumerate(samples):
#     knowledge = item['knowledge']
#     question = item['question']
#     answer = item['hallucinated_answer']  # or item['right_answer'] if you想测ground truth

# # 构造 CoT prompt
#     prompt = build_baseline_prompt(knowledge, question, answer)

#     response = requests.post(
#             "http://localhost:11434/api/generate",
#             headers={"Content-Type": "application/json"},
#             data=json.dumps({
#                 "model": "qwen3:14b",
#                 "prompt": prompt,
#                 "stream": False  # 不要流式输出
#             })
#         )

#     result = response.json()['response']
#     if 'Final Answer: 1' in result:
#       count+=1
#     elif 'Final Answer: 0' in result:
#       count0+=1
#     else:
#       print(f"Model Output:\n{result}")
#     print(f"\n====== Sample {i+1} ======")
#     # print(f"Prompt:\n{prompt}")


# # print("-"*40)
# # print(f"The accuracy of detection is {count/250}")
# # print("-"*40)
# # print(count)
# # print(count0)



# # ***********************************************************************************************************



# # import random
# # import requests
# # import json
# # import re

# # # 随机选取5条样本
# # # samples = random.sample(list(ds), 5)


# # for i, item in enumerate(samples):
# #     knowledge = item['knowledge']
# #     question = item['question']
# #     answer = item['hallucinated_answer']  # 或用 right_answer 做对比

# #     # 构造 prompt
# #     prompt = build_cot_prompt(knowledge, question, answer)

# #     # 调用 Ollama
# #     response = requests.post(
# #         "http://localhost:11434/api/generate",
# #         headers={"Content-Type": "application/json"},
# #         data=json.dumps({
# #             "model": "qwen3:14b",
# #             "prompt": prompt,
# #             "stream": False
# #         })
# #     )
# #     result = response.json()['response']

# #     print(f"\n====== Sample {i+1} ======")
# #     print(f"Model Output:\n{result}")


# 1+2 prompt strategy

In [ ]:
def build_1p2_prompt(knowledge, question, answer):
    return f"""
You are an expert hallucination detector.
Analyse the given answer based on the provided knowledge and question,
then output your decision in a fixed JSON format.

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
A: Pride and Prejudice was written by Jane Austen.
{{ "predicted_label": 0, "reason": "Author matches common knowledge." }}

example2:
Q: What is the capital city of Mars?
A: The capital of Mars is Olympus City.
{{ "predicted_label": 1, "reason": "Mars has no capital city." }}

Now is your turn:
Q: {question}
A: {answer}

At the end, reply **only** with a JSON object in the following schema
and nothing else:

{{
  "predicted_label": 0 or 1,   // 0 = factual, 1 = hallucination
  "reason": "brief explanation"
}}
"""


In [ ]:
data = ds['data']

In [ ]:
samples = data.shuffle(seed=42).select(range(250))

In [ ]:
import random, requests, json, re, time

# -------- 计时开始 --------
start = time.perf_counter()

cnt_label1 = 0          # 预测为 1 的条数
cnt_label0 = 0          # 预测为 0 的条数
bad_cases  = []         # 解析失败时存放索引 & 原始输出
TP = 0
TN = 0
FP = 0
FN = 0


# hallucinate predict
for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]      # 或 right_answer

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_1p2_prompt(knowledge, question, answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
        TP += 1
    else:
        cnt_label0 += 1
        FN += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)


# ground truth predict
for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]      # 或 right_answer

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_1p2_prompt(knowledge, question, answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
        FP += 1
    else:
        cnt_label0 += 1
        TN += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)



# -------- 计时结束 --------
elapsed = time.perf_counter() - start
print("\n=== Runtime ===")
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")

print("\n=== Confusion Matrix ===")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN))
print("Precision:",TP/(TP+FP))
print("F1:",2*TP/(2*TP+FP+FN))
print("Recall:",TP/(TP+FN))

print("=" * 40)
print(f"Predicted '1' : {cnt_label1}")
print(f"Predicted '0' : {cnt_label0}")
print(f"Parse errors  : {len(bad_cases)}")




====== Sample 1 ======

====== Sample 2 ======

====== Sample 3 ======

====== Sample 4 ======

====== Sample 5 ======

====== Sample 6 ======

====== Sample 7 ======

====== Sample 8 ======

====== Sample 9 ======

====== Sample 10 ======

====== Sample 11 ======

====== Sample 12 ======

====== Sample 13 ======

====== Sample 14 ======

====== Sample 15 ======

====== Sample 16 ======

====== Sample 17 ======

====== Sample 18 ======

====== Sample 19 ======

====== Sample 20 ======

====== Sample 21 ======

====== Sample 22 ======

====== Sample 23 ======

====== Sample 24 ======

====== Sample 25 ======

====== Sample 26 ======

====== Sample 27 ======

====== Sample 28 ======

====== Sample 29 ======

====== Sample 30 ======

====== Sample 31 ======

====== Sample 32 ======

====== Sample 33 ======

====== Sample 34 ======

====== Sample 35 ======

====== Sample 36 ======

====== Sample 37 ======

====== Sample 38 ======

====== Sample 39 ======

====== Sample 40 ======

====== S

In [ ]:
print("Accuracy:",cnt_label1/250)

Accuracy: 0.756


# 1+2+3


In [ ]:
def build_123_prompt(knowledge, question, answer):
    return f"""
You are an expert hallucination detector.
Analyse the given answer based on the provided knowledge and question,
then output your decision in a fixed JSON format.

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
Knowledge: Pride and Prejudice was written by Jane Austen.
A: Pride and Prejudice was written by Jane Austen.
{{ "predicted_label": 0, "reason": "Author matches common knowledge." }}

example2:
Q: What is the capital city of Mars?
Knowledge: Mars is planet, not a city.
A: The capital of Mars is Olympus City.
{{ "predicted_label": 1, "reason": "Mars has no capital city." }}

Now is your turn:
Q: {question}
knowledge: {knowledge}
A: {answer}

At the end, reply **only** with a JSON object in the following schema
and nothing else:

{{
  "predicted_label": 0 or 1,   // 0 = factual, 1 = hallucination
  "reason": "≤20 words brief explanation"
}}
"""


In [ ]:
data = ds['data']

In [ ]:
samples = data.shuffle(seed=42).select(range(250))

In [ ]:
import random, requests, json, re, time

# -------- 计时开始 --------
start = time.perf_counter()

cnt_label1 = 0          # 预测为 1 的条数
cnt_label0 = 0          # 预测为 0 的条数
bad_cases  = []         # 解析失败时存放索引 & 原始输出

for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]      # 或 right_answer

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_123_prompt(knowledge, question, answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
    else:
        cnt_label0 += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)

# -------- 计时结束 --------
elapsed = time.perf_counter() - start
print("*" * 40)
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")
print("-" * 40)
print(f"Predicted '1' : {cnt_label1}")
print(f"Predicted '0' : {cnt_label0}")
print(f"Parse errors  : {len(bad_cases)}")
print("+"*40)
print("Accuracy:",cnt_label1/250)



====== Sample 1 ======

====== Sample 2 ======

====== Sample 3 ======

====== Sample 4 ======

====== Sample 5 ======

====== Sample 6 ======

====== Sample 7 ======

====== Sample 8 ======

====== Sample 9 ======

====== Sample 10 ======

====== Sample 11 ======

====== Sample 12 ======

====== Sample 13 ======

====== Sample 14 ======

====== Sample 15 ======

====== Sample 16 ======

====== Sample 17 ======

====== Sample 18 ======

====== Sample 19 ======

====== Sample 20 ======

====== Sample 21 ======

====== Sample 22 ======

====== Sample 23 ======

====== Sample 24 ======

====== Sample 25 ======

====== Sample 26 ======

====== Sample 27 ======

====== Sample 28 ======

====== Sample 29 ======

====== Sample 30 ======

====== Sample 31 ======

====== Sample 32 ======

====== Sample 33 ======

====== Sample 34 ======

====== Sample 35 ======

====== Sample 36 ======

====== Sample 37 ======

====== Sample 38 ======

====== Sample 39 ======

====== Sample 40 ======

====== S

# 1+2+3+4

In [ ]:
def build_1234_prompt(knowledge, question, answer, r_answer):
    return f"""
You are an expert hallucination detector.
Analyse the given answer based on the provided knowledge and question,
then output your decision in a fixed JSON format.

Definition:
- A **hallucination** is any information in the answer that is fabricated, unverifiable, outdated, or contradicted by the knowledge or common facts.
- A **factual answer** is one where all information is fully supported by the knowledge or common facts.

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
Knowledge: Pride and Prejudice was written by Jane Austen.
factual answer: Jane Austen writted Pride and Prejudice in 1813, that is a perfect novel.
A: Pride and Prejudice was written by Jane Austen.
{{ "predicted_label": 0, "reason": "Author matches common knowledge." }}

example2:
Q: What is the capital city of Mars?
Knowledge: Mars is a planet, not a city.
factual answer: Mars is the fourth planet from the Sun. It is also known as the "Red Planet"
A: The capital of Mars is Olympus City.
{{ "predicted_label": 1, "reason": "Mars has no capital city." }}

Now is your turn:
Q: {question}
Knowledge: {knowledge}
factual answer: {r_answer}
A: {answer}

At the end, reply **only** with a JSON object in the following schema
and nothing else:

{{
  "predicted_label": 0 or 1,   // 0 = factual, 1 = hallucination
  "reason": "≤20 words brief explanation"
}}
"""


In [ ]:
data = ds['data']

In [ ]:
samples = data.shuffle(seed=42).select(range(250))

In [ ]:
import random, requests, json, re, time

# -------- 计时开始 --------
start = time.perf_counter()

cnt_label1 = 0          # 预测为 1 的条数
cnt_label0 = 0          # 预测为 0 的条数
bad_cases  = []         # 解析失败时存放索引 & 原始输出

for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]
    f_answer = item["right_answer"]

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_1234_prompt(knowledge, question, answer, f_answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
    else:
        cnt_label0 += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)

# -------- 计时结束 --------
elapsed = time.perf_counter() - start
print("*" * 40)
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")
print("-" * 40)
print(f"Predicted '1' : {cnt_label1}")
print(f"Predicted '0' : {cnt_label0}")
print(f"Parse errors  : {len(bad_cases)}")
print("+"*40)
print("Accuracy:",cnt_label1/250)



====== Sample 1 ======

====== Sample 2 ======

====== Sample 3 ======

====== Sample 4 ======

====== Sample 5 ======

====== Sample 6 ======

====== Sample 7 ======

====== Sample 8 ======

====== Sample 9 ======

====== Sample 10 ======

====== Sample 11 ======

====== Sample 12 ======

====== Sample 13 ======

====== Sample 14 ======

====== Sample 15 ======

====== Sample 16 ======

====== Sample 17 ======

====== Sample 18 ======

====== Sample 19 ======

====== Sample 20 ======

====== Sample 21 ======

====== Sample 22 ======

====== Sample 23 ======

====== Sample 24 ======

====== Sample 25 ======

====== Sample 26 ======

====== Sample 27 ======

====== Sample 28 ======

====== Sample 29 ======

====== Sample 30 ======

====== Sample 31 ======

====== Sample 32 ======

====== Sample 33 ======

====== Sample 34 ======

====== Sample 35 ======

====== Sample 36 ======

====== Sample 37 ======

====== Sample 38 ======

====== Sample 39 ======

====== Sample 40 ======

====== S

# 1+2+3+4+5

In [ ]:
def build_12345_prompt(knowledge, question, answer, r_answer):
    return f"""
You are an expert hallucination detector.
Analyse the given answer based on the provided knowledge and question,
then output your decision in a fixed JSON format.

Definition:
- A **hallucination** is any information in the answer that is fabricated, unverifiable, outdated, or contradicted by the knowledge or common facts.
- A **factual answer** is one where all information is fully supported by the knowledge or common facts.

Reasoning Instructions:
First, think step-by-step inside a <think>...</think> block to explain how you reached your decision.
Then, output ONLY a JSON object in the following format.

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
Knowledge: Pride and Prejudice was written by Jane Austen.
factual answer: Jane Austen writted Pride and Prejudice in 1813, that is a perfect novel.
A: Pride and Prejudice was written by Jane Austen.
<think>The answer correctly matches the known fact provided in the knowledge. No conflicting or fabricated info.</think>
{{ "predicted_label": 0, "reason": "Author matches common knowledge." }}

example2:
Q: What is the capital city of Mars?
Knowledge: Mars is a planet, not a city.
factual answer: Mars is the fourth planet from the Sun. It is also known as the "Red Planet"
A: The capital of Mars is Olympus City.
<think>Mars is not a country, so it cannot have a capital city. The answer is clearly made up.</think>
{{ "predicted_label": 1, "reason": "Mars has no capital city." }}

Now is your turn:
Q: {question}
Knowledge: {knowledge}
factual answer: {r_answer}
A: {answer}

At the end, follow this format:
1. A step-by-step reasoning inside <think> ... </think>
2. Then the JSON output:

{{
  "predicted_label": 0 or 1,   // 0 = factual, 1 = hallucination
  "reason": "≤20 words brief explanation"
}}
"""


In [ ]:
data = ds['data']

In [ ]:
samples = data.shuffle(seed=42).select(range(250))

In [ ]:
import random, requests, json, re, time

# -------- 计时开始 --------
start = time.perf_counter()

cnt_label1 = 0          # 预测为 1 的条数
cnt_label0 = 0          # 预测为 0 的条数
bad_cases  = []         # 解析失败时存放索引 & 原始输出

for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]
    f_answer = item["right_answer"]

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_12345_prompt(knowledge, question, answer, f_answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
    else:
        cnt_label0 += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)

# -------- 计时结束 --------
elapsed = time.perf_counter() - start
print("*" * 40)
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")
print("-" * 40)
print(f"Predicted '1' : {cnt_label1}")
print(f"Predicted '0' : {cnt_label0}")
print(f"Parse errors  : {len(bad_cases)}")
print("+"*40)
print("Accuracy:",cnt_label1/250)



====== Sample 1 ======

====== Sample 2 ======

====== Sample 3 ======

====== Sample 4 ======

====== Sample 5 ======

====== Sample 6 ======

====== Sample 7 ======

====== Sample 8 ======

====== Sample 9 ======

====== Sample 10 ======

====== Sample 11 ======

====== Sample 12 ======

====== Sample 13 ======

====== Sample 14 ======

====== Sample 15 ======

====== Sample 16 ======

====== Sample 17 ======

====== Sample 18 ======

====== Sample 19 ======

====== Sample 20 ======

====== Sample 21 ======

====== Sample 22 ======

====== Sample 23 ======

====== Sample 24 ======

====== Sample 25 ======

====== Sample 26 ======

====== Sample 27 ======

====== Sample 28 ======

====== Sample 29 ======

====== Sample 30 ======

====== Sample 31 ======

====== Sample 32 ======

====== Sample 33 ======

====== Sample 34 ======

====== Sample 35 ======

====== Sample 36 ======

====== Sample 37 ======

====== Sample 38 ======

====== Sample 39 ======

====== Sample 40 ======

====== S

# 1+2+3+4+5+6

In [ ]:
def build_123456_prompt(knowledge, question, answer, r_answer):
    return f"""
You are an expert hallucination detector.
Analyse the given answer based on the provided knowledge and question,
then output your decision in a fixed JSON format.

Definition:
- A **hallucination** is any information in the answer that is fabricated, unverifiable, outdated, or contradicted by the knowledge or common facts.
- A **factual answer** is one where all information is fully supported by the knowledge or common facts.

Reasoning Instructions:
First, think step-by-step inside a <think>...</think> block to explain how you reached your decision.
Then, output ONLY a JSON object in the following format.

There are such known Hallucination Types may exist in the answer:
- **Extrinsic-Hard**: Contradicts known facts.
- **Extrinsic-Soft**: Cannot be verified based on given or common knowledge.
- **Intrinsic**: Contradicts the question or previous context (e.g. internal inconsistency).

There are 2 examples:

example1:
Q: Who wrote “Pride and Prejudice”?
Knowledge: Pride and Prejudice was written by Jane Austen.
factual answer: Jane Austen writted Pride and Prejudice in 1813, that is a perfect novel.
A: Pride and Prejudice was written by Jane Austen.
<think>The answer correctly matches the known fact provided in the knowledge. No conflicting or fabricated info.</think>
{{ "predicted_label": 0, "reason": "Author matches common knowledge." }}

example2:
Q: What is the capital city of Mars?
Knowledge: Mars is a planet, not a city.
factual answer: Mars is the fourth planet from the Sun. It is also known as the "Red Planet"
A: The capital of Mars is Olympus City.
<think>Mars is not a country, so it cannot have a capital city. The answer is clearly made up.</think>
{{ "predicted_label": 1, "reason": "Mars has no capital city." }}

Now is your turn:
Q: {question}
Knowledge: {knowledge}
factual answer: {r_answer}
A: {answer}

At the end, follow this format:
1. A step-by-step reasoning inside <think> ... </think>
2. Then the JSON output:

{{
  "predicted_label": 0 or 1,   // 0 = factual, 1 = hallucination
  "reason": "≤20 words brief explanation"
}}
"""


In [ ]:
data = ds['data']

In [ ]:
samples = data.shuffle(seed=42).select(range(250))

In [ ]:
import random, requests, json, re, time

# -------- 计时开始 --------
start = time.perf_counter()

cnt_label1 = 0          # 预测为 1 的条数
cnt_label0 = 0          # 预测为 0 的条数
bad_cases  = []         # 解析失败时存放索引 & 原始输出

for i, item in enumerate(samples):
    knowledge = item["knowledge"]
    question  = item["question"]
    answer    = item["hallucinated_answer"]
    f_answer = item["right_answer"]

    # === 构造 prompt（你已有 build_1p2_prompt） ===
    prompt = build_123456_prompt(knowledge, question, answer, f_answer)

    # === 调用 Ollama ===
    resp = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model":  "qwen3:14b",
            "prompt": prompt,
            "stream": False
        }),
        timeout=120
    )

    result_text = resp.json().get("response", "")
    # ---------------------------------------------
    # 1️⃣ 抓出 JSON 片段 —— 容忍模型偶尔外包装 ```json … ```
    # ---------------------------------------------
    match = re.search(r"\{.*?\}", result_text, re.S)
    if not match:
        print(f"[Parse ⚠️ ] No JSON found in sample {i+1}")
        bad_cases.append((i, result_text))
        continue

    try:
        pred_obj = json.loads(match.group())
        label    = int(pred_obj["predicted_label"])
    except Exception as e:
        print(f"[Parse ⚠️ ] JSON load error in sample {i+1}: {e}")
        bad_cases.append((i, result_text))
        continue

    # ---------------------------------------------
    # 2️⃣ 统计
    # ---------------------------------------------
    if label == 1:
        cnt_label1 += 1
    else:
        cnt_label0 += 1

    # —— 如需打印进度，可放开下面两行 ——
    print(f"\n====== Sample {i+1} ======")
    # print(result_text)

# -------- 计时结束 --------
elapsed = time.perf_counter() - start
print("*" * 40)
print(f"Total run-time : {elapsed:.2f} s  "
      f"({elapsed/len(samples):.3f} s / sample)")
print("-" * 40)
print(f"Predicted '1' : {cnt_label1}")
print(f"Predicted '0' : {cnt_label0}")
print(f"Parse errors  : {len(bad_cases)}")
print("+"*40)
print("Accuracy:",cnt_label1/250)



====== Sample 1 ======

====== Sample 2 ======

====== Sample 3 ======

====== Sample 4 ======

====== Sample 5 ======

====== Sample 6 ======

====== Sample 7 ======

====== Sample 8 ======

====== Sample 9 ======

====== Sample 10 ======

====== Sample 11 ======

====== Sample 12 ======

====== Sample 13 ======

====== Sample 14 ======

====== Sample 15 ======

====== Sample 16 ======

====== Sample 17 ======

====== Sample 18 ======

====== Sample 19 ======

====== Sample 20 ======

====== Sample 21 ======

====== Sample 22 ======

====== Sample 23 ======

====== Sample 24 ======

====== Sample 25 ======

====== Sample 26 ======

====== Sample 27 ======

====== Sample 28 ======

====== Sample 29 ======

====== Sample 30 ======

====== Sample 31 ======

====== Sample 32 ======

====== Sample 33 ======

====== Sample 34 ======

====== Sample 35 ======

====== Sample 36 ======

====== Sample 37 ======

====== Sample 38 ======

====== Sample 39 ======

====== Sample 40 ======

====== S

# 1+2+3+4+5+6+7